In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\HP\Desktop\ML_HACKATHON\datasets\train.csv")
df.head(5)

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [4]:
df.isna().sum()

image_link      0
group_id        0
entity_name     0
entity_value    0
dtype: int64

In [5]:
df.size

1055436

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [7]:
# !pip install constants

In [8]:
import re
# import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

In [9]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [10]:
# def common_mistake(unit):
#     if unit in constants.allowed_units:
#         return unit
#     if unit.replace('ter', 'tre') in constants.allowed_units:
#         return unit.replace('ter', 'tre')
#     if unit.replace('feet', 'foot') in constants.allowed_units:
#         return unit.replace('feet', 'foot')
#     return unit

# def parse_string(s):
#     s_stripped = "" if s==None or str(s)=='nan' else s.strip()
#     if s_stripped == "":
#         return None, None
#     pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
#     if not pattern.match(s_stripped):
#         raise ValueError("Invalid format in {}".format(s))
#     parts = s_stripped.split(maxsplit=1)
#     number = float(parts[0])
#     unit = common_mistake(parts[1])
#     if unit not in constants.allowed_units:
#         raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
#             unit, s, constants.allowed_units))
#     return number, unit


# def create_placeholder_image(image_save_path):
#     try:
#         placeholder_image = Image.new('RGB', (100, 100), color='black')
#         placeholder_image.save(image_save_path)
#     except Exception as e:
#         return

# def download_image(image_link, save_folder, retries=3, delay=3):
#     if not isinstance(image_link, str):
#         return

#     filename = Path(image_link).name
#     image_save_path = os.path.join(save_folder, filename)

#     if os.path.exists(image_save_path):
#         return

#     for _ in range(retries):
#         try:
#             urllib.request.urlretrieve(image_link, image_save_path)
#             return
#         except:
#             time.sleep(delay)
    
#     create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images
    
# def download_images(image_links, download_folder, allow_multiprocessing=True):
#     if not os.path.exists(download_folder):
#         os.makedirs(download_folder)

#     if allow_multiprocessing:
#         download_image_partial = partial(
#             download_image, save_folder=download_folder, retries=3, delay=3)

#         with multiprocessing.Pool(60) as pool:
#             list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
#             pool.close()
#             pool.join()
#     else:
#         for image_link in tqdm(image_links, total=len(image_links)):
#             download_image(image_link, save_folder=download_folder, retries=3, delay=3)

In [11]:
# import os
# import urllib.request
# from tqdm import tqdm
# from pathlib import Path
# from PIL import Image
# import time
# from concurrent.futures import ThreadPoolExecutor

# # Function to create a black placeholder image in case of failure
# def create_placeholder_image(image_save_path):
#     try:
#         placeholder_image = Image.new('RGB', (100, 100), color='black')
#         placeholder_image.save(image_save_path)
#     except Exception as e:
#         return

# # Function to download a single image
# def download_image(image_link, save_folder, retries=3, delay=3):
#     if not isinstance(image_link, str):
#         return

#     filename = Path(image_link).name
#     image_save_path = os.path.join(save_folder, filename)

#     if os.path.exists(image_save_path):
#         return

#     for _ in range(retries):
#         try:
#             urllib.request.urlretrieve(image_link, image_save_path)
#             return
#         except:
#             time.sleep(delay)
    
#     # Create a placeholder if the image fails to download
#     create_placeholder_image(image_save_path)

# # Function to download images using ThreadPoolExecutor (works in Jupyter)
# def download_images(image_links, download_folder, max_workers=10):
#     if not os.path.exists(download_folder):
#         os.makedirs(download_folder)

#     with ThreadPoolExecutor(max_workers=max_workers) as executor:
#         list(tqdm(executor.map(lambda link: download_image(link, download_folder), image_links), total=len(image_links)))

# # Example usage in Jupyte
# download_folder = "C:/Users/HP/Desktop/ML_HACKATHON/imgs"
# image_links = df['image_link'][:500].dropna().tolist()

# # Download images using ThreadPoolExecutor
# download_images(image_links, download_folder, max_workers=10)


In [12]:
# import numpy as np
# from PIL import Image
# import os
# from tqdm import tqdm
# from pathlib import Path

# # Define the folder where the processed images will be saved
# processed_folder = r"C:\Users\HP\Desktop\ML_HACKATHON\imgs\preimg"  # Use raw string for paths

# # Create the output folder if it does not exist
# if not os.path.exists(processed_folder):
#     os.makedirs(processed_folder)

# # Function to preprocess a single image: resize and normalize
# def preprocess_image(image_path, output_path, target_size=(224, 224)):
#     try:
#         # Open the image
#         image = Image.open(image_path)
#         # Resize the image to the target size
#         image = image.resize(target_size)
#         # Convert the image to a numpy array and normalize pixel values to [0, 1]
#         image_array = np.array(image) / 255.0
#         # Convert the array back to an image and save it
#         processed_image = Image.fromarray((image_array * 255).astype(np.uint8))
#         processed_image.save(output_path)
#     except Exception as e:
#         print(f"Failed to process image {image_path}: {e}")

# # Set the download folder where images are located
# download_folder = "C:/Users/HP/Desktop/ML_HACKATHON/imgs"

# # List all images in the downloaded folder with common extensions
# image_paths = list(Path(download_folder).glob("*.jpg")) + \
#               list(Path(download_folder).glob("*.jpeg")) + \
#               list(Path(download_folder).glob("*.png"))  # Adjust patterns for other extensions as needed

# # Preprocess and save each image
# for image_path in tqdm(image_paths, desc="Processing Images"):
#     # Define the output path
#     output_path = os.path.join(processed_folder, image_path.name)
#     preprocess_image(image_path, output_path)

# print("Image preprocessing complete.")


In [13]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip install fuzzywuzzy


In [15]:
# import pytesseract
# from PIL import Image, ImageEnhance, ImageFilter, ImageOps
# import re
# import os
# from tqdm import tqdm
# from pathlib import Path
# import pandas as pd
# from fuzzywuzzy import fuzz

# # Specify the path to the Tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Update this path as needed

# # Folder containing the original downloaded images
# download_folder = "C:/Users/HP/Desktop/ML_HACKATHON/imgs"

# # Load the training data from train.csv
# train_csv_path = "train.csv"  # Update path as needed
# train_data = pd.read_csv(train_csv_path)

# # Enhanced OCR function with additional preprocessing steps
# def extract_text_from_image(image_path):
#     try:
#         # Open the image and convert to grayscale
#         image = Image.open(image_path).convert('L')
        
#         # Advanced preprocessing: Resize, invert colors, binarize, and enhance
#         image = image.resize((600, 600))  # Larger resize to capture small text
#         image = ImageOps.invert(image)  # Invert colors to improve contrast
#         image = ImageEnhance.Contrast(image).enhance(3)  # Further increase contrast
#         image = image.filter(ImageFilter.SHARPEN)  # Apply sharpening filter
#         image = image.point(lambda x: 0 if x < 150 else 255, '1')  # Fine-tune binarization

#         # Tesseract configuration for optimal OCR performance
#         custom_config = r'--oem 3 --psm 6'  # Test with other modes like psm 3, 11 for different layouts
#         text = pytesseract.image_to_string(image, config=custom_config)
#         return text
#     except Exception as e:
#         print(f"Failed to extract text from {image_path}: {e}")
#         return ""

# # Function to parse key entity values from the extracted text
# def parse_entity_values(text):
#     # Regular expressions for matching weight, volume, and dimensions
#     weight_pattern = r'(\d+(\.\d+)?\s?(kg|gram|g|mg|pound|lb|ounce|oz))'
#     volume_pattern = r'(\d+(\.\d+)?\s?(ml|l|liter|cup|fluid ounce|fl oz))'
#     dimension_pattern = r'(\d+(\.\d+)?\s?(cm|mm|m|inch|in|feet|ft))'

#     weights = re.findall(weight_pattern, text, re.IGNORECASE)
#     volumes = re.findall(volume_pattern, text, re.IGNORECASE)
#     dimensions = re.findall(dimension_pattern, text, re.IGNORECASE)

#     # Extracting and formatting results
#     extracted_values = {
#         "weights": [w[0] for w in weights],
#         "volumes": [v[0] for v in volumes],
#         "dimensions": [d[0] for d in dimensions]
#     }
#     return extracted_values

# # Fuzzy matching function to improve alignment
# def fuzzy_match(expected, extracted_values):
#     best_match = None
#     highest_score = 0
    
#     for key, values in extracted_values.items():
#         for value in values:
#             # Calculate the match score between the expected value and extracted value
#             score = fuzz.partial_ratio(expected.lower(), value.lower())
#             if score > highest_score:
#                 highest_score = score
#                 best_match = value
    
#     # Consider matches with a high enough score as valid
#     return best_match if highest_score > 70 else None

# # List all images in the download folder
# downloaded_image_paths = list(Path(download_folder).glob("*.jpg"))  # Adjust extension if different

# # Dictionary to store OCR results
# ocr_results = {}

# # Perform OCR and extract entity values for each image
# for image_path in tqdm(downloaded_image_paths, desc="Extracting OCR"):
#     # Extract text from the image
#     extracted_text = extract_text_from_image(image_path)
#     # Parse key entity values from the extracted text
#     entity_values = parse_entity_values(extracted_text)
#     # Store the results
#     ocr_results[image_path.name] = entity_values

# # Align extracted text with training labels from train.csv
# alignment_results = []

# for index, row in train_data.iterrows():
#     image_name = os.path.basename(row['image_link'])  # Extract the image filename from the path
#     entity_name = row['entity_name']
#     entity_value = row['entity_value']
    
#     # Retrieve OCR results for the corresponding image
#     if image_name in ocr_results:  # Check if the image name is in OCR results
#         extracted_values = ocr_results[image_name]
        
#         # Use fuzzy matching to find the best match
#         matched_value = fuzzy_match(entity_value, extracted_values)
        
#         alignment_results.append({
#             'image_name': image_name,
#             'entity_name': entity_name,
#             'expected_value': entity_value,
#             'extracted_value': matched_value if matched_value else "No match found"
#         })

# # Convert results to DataFrame for easier inspection
# alignment_df = pd.DataFrame(alignment_results)

# # Display some results
# print(alignment_df.head())

# # Save results to a CSV file if needed
# alignment_df.to_csv("alignment_results.csv", index=False)

# print("OCR extraction, entity value parsing, and alignment complete.")


In [16]:
# df['entity_name'].value_counts()

In [17]:
# pip install tensorflow#


In [18]:
# # Necessary imports (ensure these are included in your environment setup)
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dense, Flatten
# from tensorflow.keras.models import Model
# from tensorflow.keras.applications import VGG16, EfficientNetB0
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# import cv2
# import numpy as np
# import os
# from tqdm import tqdm


In [19]:
# # Define U-Net Model with VGG16 Backbone
# def unet_with_vgg16(input_shape=(256, 256, 3)):
#     vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
#     inputs = vgg16.input
#     c1 = vgg16.get_layer("block1_conv2").output
#     p1 = MaxPooling2D((2, 2))(c1)
    
#     c2 = vgg16.get_layer("block2_conv2").output
#     p2 = MaxPooling2D((2, 2))(c2)
    
#     c3 = vgg16.get_layer("block3_conv3").output
#     p3 = MaxPooling2D((2, 2))(c3)
    
#     c4 = vgg16.get_layer("block4_conv3").output
#     p4 = MaxPooling2D((2, 2))(c4)
    
#     c5 = vgg16.get_layer("block5_conv3").output
    
#     u6 = UpSampling2D((2, 2))(c5)
#     u6 = concatenate([u6, c4])
#     c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
#     c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    
#     u7 = UpSampling2D((2, 2))(c6)
#     u7 = concatenate([u7, c3])
#     c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
#     c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    
#     u8 = UpSampling2D((2, 2))(c7)
#     u8 = concatenate([u8, c2])
#     c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
#     c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    
#     u9 = UpSampling2D((2, 2))(c8)
#     u9 = concatenate([u9, c1])
#     c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
#     c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    
#     outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
#     model = Model(inputs=[inputs], outputs=[outputs])
#     return model

# # Compile U-Net Model
# input_shape = (256, 256, 3)
# unet_model = unet_with_vgg16(input_shape)
# unet_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# # Train U-Net Model (Ensure data directories are set correctly)
# train_image_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2)
# train_mask_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2)

# train_image_generator = train_image_datagen.flow_from_directory(
#     'train_images/',  # Update with your image directory
#     class_mode=None,
#     target_size=input_shape[:2],
#     batch_size=8,
#     seed=42)

# train_mask_generator = train_mask_datagen.flow_from_directory(
#     'train_masks/',  # Update with your mask directory
#     class_mode=None,
#     target_size=input_shape[:2],
#     batch_size=8,
#     seed=42)

# train_generator = zip(train_image_generator, train_mask_generator)

# # Train U-Net Model
# unet_model.fit(
#     train_generator,
#     steps_per_epoch=len(train_image_generator),
#     epochs=20,
#     callbacks=[
#         ModelCheckpoint("unet_best_model.h5", save_best_only=True),
#         EarlyStopping(patience=5, restore_best_weights=True)
#     ])


In [20]:
!pip install tensorflow


In [21]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, precision_score, recall_score
# import cv2
# import requests
# from io import BytesIO
# from PIL import Image
# import os

# # Define the correct file path
# file_path = r"C:\Users\HP\Desktop\ML_HACKATHON\train.csv"

# # Check if the file exists and load data
# if os.path.exists(file_path):
#     train_df = pd.read_csv(file_path)
#     print("File loaded successfully.")
# else:
#     raise FileNotFoundError(f"File not found at path: {file_path}")

# # Helper function to download and preprocess images
# def preprocess_image(url, target_size=(224, 224)):
#     try:
#         response = requests.get(url)
#         img = Image.open(BytesIO(response.content))
#         img = img.resize(target_size)
#         img = np.array(img) / 255.0  # Normalize
#         return img
#     except Exception as e:
#         print(f"Error processing image at {url}: {e}")
#         return np.zeros((224, 224, 3))  # Return a blank image on failure

# # Preprocess images and labels
# train_df['processed_image'] = train_df['image_link'].apply(preprocess_image)
# X = np.stack(train_df['processed_image'])
# y = train_df['entity_value'].astype(str)  # Convert labels to string format

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Image data generator for augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # EfficientNetB0 with transfer learning
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)  # Adding dropout for regularization
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(1, activation='sigmoid')(x)  # Binary classification, adjust activation if multi-class
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(
#     datagen.flow(X_train, y_train, batch_size=32),
#     validation_data=(X_val, y_val),
#     epochs=20,
#     steps_per_epoch=len(X_train) // 32,
#     validation_steps=len(X_val) // 32,
#     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
# )

# # Evaluate model performance
# val_predictions = model.predict(X_val).round().astype(int)
# f1 = f1_score(y_val, val_predictions, average='weighted')
# precision = precision_score(y_val, val_predictions, average='weighted')
# recall = recall_score(y_val, val_predictions, average='weighted')

# print(f"Validation F1 Score: {f1}")
# print(f"Validation Precision: {precision}")
# print(f"Validation Recall: {recall}")

# # Load test data and preprocess
# test_file_path = r"C:\Users\HP\Desktop\ML_HACKATHON\test.csv"
# if os.path.exists(test_file_path):
#     test_df = pd.read_csv(test_file_path)
#     test_df['processed_image'] = test_df['image_link'].apply(preprocess_image)
#     X_test = np.stack(test_df['processed_image'])
# else:
#     raise FileNotFoundError(f"Test file not found at path: {test_file_path}")

# # Generate predictions for test data
# test_predictions = model.predict(X_test).round().astype(int)  # Adjust based on your task

# # Format predictions for submission
# test_df['prediction'] = [f"{pred} unit" for pred in test_predictions]  # Adjust the format as per requirements
# output = test_df[['index', 'prediction']]
# output.to_csv('test_out.csv', index=False)

# # Run sanity check (if available)
# # Assuming 'sanity.py' is correctly set up to validate the output format
# print("Output saved as 'test_out.csv'. Run sanity checks to ensure format validity.")


In [22]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, precision_score, recall_score
# import cv2
# import requests
# from io import BytesIO
# from PIL import Image
# import os

# # Define the correct file paths
# train_file_path = r"C:\Users\HP\Desktop\ML_HACKATHON\train.csv"
# test_file_path = r"C:\Users\HP\Desktop\ML_HACKATHON\test.csv"

# # Load training data
# if os.path.exists(train_file_path):
#     train_df = pd.read_csv(train_file_path)
#     print("Training file loaded successfully.")
# else:
#     raise FileNotFoundError(f"Training file not found at path: {train_file_path}")

# # Helper function to download and preprocess images
# def preprocess_image(url, target_size=(224, 224)):
#     try:
#         response = requests.get(url)
#         img = Image.open(BytesIO(response.content)).convert('RGB')
#         img = img.resize(target_size)
#         img = np.array(img) / 255.0  # Normalize
#         return img
#     except Exception as e:
#         print(f"Error processing image at {url}: {e}")
#         return np.zeros((224, 224, 3))  # Return a blank image on failure

# # Preprocess images and labels
# train_df['processed_image'] = train_df['image_link'].apply(preprocess_image)
# X = np.stack(train_df['processed_image'])
# y = train_df['entity_value'].astype(str)  # Convert labels to string format

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Image data generator for augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # EfficientNetB0 with transfer learning
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)  # Adding dropout for regularization
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(1, activation='sigmoid')(x)  # Adjust for binary classification; modify for multi-class if needed
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(
#     datagen.flow(X_train, y_train, batch_size=32),
#     validation_data=(X_val, y_val),
#     epochs=20,
#     steps_per_epoch=len(X_train) // 32,
#     validation_steps=len(X_val) // 32,
#     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
# )

# # Evaluate model performance
# val_predictions = model.predict(X_val).round().astype(int)
# f1 = f1_score(y_val, val_predictions, average='weighted')
# precision = precision_score(y_val, val_predictions, average='weighted')
# recall = recall_score(y_val, val_predictions, average='weighted')

# print(f"Validation F1 Score: {f1}")
# print(f"Validation Precision: {precision}")
# print(f"Validation Recall: {recall}")

# # Load and preprocess test data
# if os.path.exists(test_file_path):
#     test_df = pd.read_csv(test_file_path)
#     test_df['processed_image'] = test_df['image_link'].apply(preprocess_image)
#     X_test = np.stack(test_df['processed_image'])
#     print("Test file loaded and preprocessed successfully.")
# else:
#     raise FileNotFoundError(f"Test file not found at path: {test_file_path}")

# # Generate predictions for test data
# test_predictions = model.predict(X_test).round().astype(int)  # Adjust based on your task
# test_df['prediction'] = [f"{pred[0]} unit" for pred in test_predictions]  # Format predictions as required

# # Save output in the required format
# output = test_df[['index', 'prediction']]
# output.to_csv('test_out.csv', index=False)
# print("Output saved as 'test_out.csv'. Run sanity checks to ensure format validity.")

# # Optionally, run sanity checks using provided scripts
# # Ensure the output is formatted correctly before submission


In [23]:
# import subprocess

# # Define the command to run the sanity check script
# sanity_check_script = r"sanity.py"  # Ensure this is the correct path to your sanity check script
# output_file = 'test_out.csv'  # Path to the output file you want to check

# # Run the sanity check
# try:
#     result = subprocess.run(
#         ['python', sanity_check_script, output_file],
#         check=True,
#         capture_output=True,
#         text=True
#     )
#     print("Sanity check output:")
#     print(result.stdout)
# except subprocess.CalledProcessError as e:
#     print("Sanity check failed:")
#     print(e.output)
#     print(e.stderr)


In [24]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, concatenate, Dropout, UpSampling2D

def unet_with_pretrained_encoder(input_shape=(256, 256, 3)):
    # Load a pre-trained model as the encoder
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Use the layers of the pre-trained model
    encoder_layers = [base_model.get_layer(name).output for name in ['block1_pool', 'block2_pool', 'block3_pool', 'block4_pool']]
    
    # Define the input layer
    inputs = base_model.input
    
    # Decoder part of the U-Net
    x = encoder_layers[-1]  # Start from the deepest layer
    
    # Upsampling and concatenating with corresponding encoder layers
    for i in range(3, 0, -1):
        x = Conv2DTranspose(256 // (2 ** i), (3, 3), strides=(2, 2), padding='same')(x)
        x = concatenate([x, encoder_layers[i-1]])
        x = Conv2D(256 // (2 ** i), (3, 3), activation='relu', padding='same')(x)
        x = Dropout(0.2)(x)

    # Final layer to match the mask output
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

    # Create the model
    model = Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Instantiate the model
unet_model = unet_with_pretrained_encoder()


In [25]:
# import cv2
# import numpy as np
# import os
# import pandas as pd
# from tqdm import tqdm  # For progress bar

# # Function to load images from a file path and prepare them for the model
# def load_and_preprocess_image(image_path, target_size=(256, 256)):
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Error loading image: {image_path}")
#         return None
#     image = cv2.resize(image, target_size)
#     image = image / 255.0  # Normalize the image
#     return image

# # Function to generate masks using the U-Net model
# def generate_masks(unet_model, csv_file, image_directory, output_directory, target_size=(256, 256)):
#     # Read the CSV file
#     df = pd.read_csv(csv_file)
    
#     # Ensure the output directory exists
#     if not os.path.exists(output_directory):
#         os.makedirs(output_directory)
        
#     # Iterate through the image paths in the CSV
#     for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Images"):
#         image_link = row['image_link']
#         image_filename = os.path.basename(image_link)  # Extract the image filename from the URL
#         image_path = os.path.join(image_directory, image_filename)
        
#         # Check if the image exists locally
#         if os.path.exists(image_path):
#             # Load and preprocess the image
#             image = load_and_preprocess_image(image_path, target_size)
#             if image is None:
#                 continue
            
#             image = np.expand_dims(image, axis=0)  # Add batch dimension

#             # Predict mask
#             predicted_mask = unet_model.predict(image)[0]

#             # Convert prediction to binary mask
#             binary_mask = (predicted_mask > 0.5).astype(np.uint8) * 255

#             # Save mask with the same filename as the input image
#             mask_filename = os.path.splitext(image_filename)[0] + '.png'  # Save as PNG
#             mask_path = os.path.join(output_directory, mask_filename)
#             cv2.imwrite(mask_path, binary_mask)
#         else:
#             print(f"Image not found: {image_path}")

# # Use the function to generate masks
# # Ensure 'train.csv' has a column named 'image_link' with the image filenames
# generate_masks(unet_model, 'train.csv', "C:/Users/HP/Desktop/ML_HACKATHON/imgs", 'train_masks/')










In [26]:
import cv2
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import requests
from pathlib import Path
import time

# Define the new download folder path
download_folder = "C:/Users/HP/Desktop/ML_HACKATHON/newimgg"

# Function to create a placeholder image
def create_placeholder_image(image_save_path):
    try:
        from PIL import Image
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        print(f"Error creating placeholder image: {e}")

# Function to download images from URLs
def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            response = requests.get(image_link, stream=True)
            if response.status_code == 200:
                with open(image_save_path, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                return
        except Exception as e:
            print(f"Error downloading {image_link}: {e}")
            time.sleep(delay)
    
    create_placeholder_image(image_save_path)  # Create a black placeholder image for invalid links/images

# Function to download the top 100 images
def download_top_images(csv_file, download_folder, top_n=100):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Ensure the download directory exists
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    
    # Get the top 100 image links
    image_links = df['image_link'].head(top_n)
    
    # Download images
    for image_link in tqdm(image_links, desc="Downloading Images"):
        download_image(image_link, download_folder)

# Function to load images from a file path and prepare them for the model
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize the image
    return image

# Function to generate masks using the U-Net model
def generate_masks(unet_model, image_directory, output_directory, target_size=(256, 256)):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    # Iterate through the images in the directory
    for image_filename in tqdm(os.listdir(image_directory), desc="Generating Masks"):
        image_path = os.path.join(image_directory, image_filename)
        
        # Check if the image exists locally
        if os.path.exists(image_path):
            # Load and preprocess the image
            image = load_and_preprocess_image(image_path, target_size)
            if image is None:
                continue
            
            image = np.expand_dims(image, axis=0)  # Add batch dimension

            # Predict mask
            predicted_mask = unet_model.predict(image)[0]

            # Convert prediction to binary mask
            binary_mask = (predicted_mask > 0.5).astype(np.uint8) * 255

            # Save mask with the same filename as the input image
            mask_filename = os.path.splitext(image_filename)[0] + '.png'  # Save as PNG
            mask_path = os.path.join(output_directory, mask_filename)
            cv2.imwrite(mask_path, binary_mask)
        else:
            print(f"Image not found: {image_path}")

# Step 1: Download the top 100 images from the CSV file to the specified folder
download_top_images('train.csv', download_folder, top_n=100)

# Step 2: Generate masks for the downloaded images
generate_masks(unet_model, download_folder, 'train_masks/')


Generating Masks:   0%|                                                                         | 0/99 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


Generating Masks:   1%|▋                                                                | 1/99 [00:01<02:45,  1.69s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step


Generating Masks:   2%|█▎                                                               | 2/99 [00:02<01:47,  1.11s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step


Generating Masks:  98%|██████████████████████████████████████████████████████████████▋ | 97/99 [01:13<00:01,  1.17it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step


Generating Masks: 100%|████████████████████████████████████████████████████████████████| 99/99 [01:13<00:00,  1.34it/s]

Error loading image: C:/Users/HP/Desktop/ML_HACKATHON/newimgg\class


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = UpSampling2D((2, 2))(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Instantiate the U-Net model
unet_model = unet_model(input_size=(256, 256, 3))
unet_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 256, 256, 64)      │           1,792 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 256, 256, 64)      │          36,928 │ conv2d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 128, 128, 64)      │               0 │ conv2d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_6 (Conv2D)             │ (None, 128, 128, 128)     │          73,856 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_7 (Conv2D)             │ (None, 128, 128, 128)     │         147,584 │ conv2d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 64, 64, 128)       │               0 │ conv2d_7[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_8 (Conv2D)             │ (None, 64, 64, 256)       │         295,168 │ max_pooling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_9 (Conv2D)             │ (None, 64, 64, 256)       │         590,080 │ conv2d_8[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 32, 32, 256)       │               0 │ conv2d_9[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_10 (Conv2D)            │ (None, 32, 32, 512)       │       1,180,160 │ max_pooling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_11 (Conv2D)            │ (None, 32, 32, 512)       │       2,359,808 │ conv2d_10[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 16, 16, 512)       │               0 │ conv2d_11[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_12 (Conv2D)            │ (None, 16, 16, 1024)      │       4,719,616 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_13 (Conv2D)            │ (None, 16, 16, 1024)      │       9,438,208 │ conv2d_12[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d (UpSampling2D)  │ (None, 32, 32, 1024)      │               

 Total params: 31,378,945 (119.70 MB)

 Trainable params: 31,378,945 (119.70 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define a custom generator to use U-Net for generating masks on the fly
def image_generator_with_generated_masks(model, image_folder, batch_size, target_size=(256, 256)):
    # Define the ImageDataGenerator for images
    image_datagen = ImageDataGenerator(rescale=1./255)

    # Load images from the directory, ensuring the correct subdirectory structure
    image_generator = image_datagen.flow_from_directory(
        image_folder, 
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,  # No labels needed, just images
        seed=42,
        color_mode='rgb'  # Assuming images are in RGB format
    )

    while True:
        # Use __next__() to get the next batch of images
        images = image_generator.__next__()
        generated_masks = []

        # Generate masks using the model
        for img in images:
            img_array = np.expand_dims(img, axis=0)  # Add batch dimension
            predicted_mask = model.predict(img_array)  # Generate mask
            predicted_mask = (predicted_mask > 0.5).astype(np.float32)  # Threshold the mask
            generated_masks.append(predicted_mask[0])  # Remove batch dimension

        yield images, np.array(generated_masks)  # Yield images and generated masks as a tuple

# Define paths
download_folder = "C:/Users/HP/Desktop/ML_HACKATHON/newimgg"  # Path to images with subdirectory 'class1'

# Define batch size
batch_size = 8

# Ensure that 'unet_model' is defined and compiled before this point
# Example: unet_model = ... (your U-Net model initialization code here)

# Initialize the custom generator
train_generator = image_generator_with_generated_masks(unet_model, download_folder, batch_size)

# Verify generator output before training
import traceback

try:
    sample_images, sample_masks = next(train_generator)
    print(f"Sample images shape: {sample_images.shape}")
    print(f"Sample masks shape: {sample_masks.shape}")
except Exception as e:
    print(f"Error in generator output: {e}")
    traceback.print_exc()

# Check that the subdirectory 'class1' exists and contains images
steps_per_epoch = max(1, len(os.listdir(os.path.join(download_folder, 'class1'))) // batch_size)

# Train the U-Net model using generated masks directly
unet_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,  # Ensure this matches the number of available images
    epochs=5,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint("unet_best_weights.weights.h5", save_best_only=True, save_weights_only=True),  # Save weights only with correct extension
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ]
)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

# def unet_model(input_size=(256, 256, 3)):
#     inputs = Input(input_size)

#     # Encoder
#     c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
#     c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
#     p1 = MaxPooling2D((2, 2))(c1)

#     c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
#     c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
#     p2 = MaxPooling2D((2, 2))(c2)

#     c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
#     c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
#     p3 = MaxPooling2D((2, 2))(c3)

#     c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
#     c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
#     p4 = MaxPooling2D((2, 2))(c4)

#     # Bottleneck
#     c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
#     c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

#     # Decoder
#     u6 = UpSampling2D((2, 2))(c5)
#     u6 = Concatenate()([u6, c4])
#     c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
#     c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

#     u7 = UpSampling2D((2, 2))(c6)
#     u7 = Concatenate()([u7, c3])
#     c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
#     c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

#     u8 = UpSampling2D((2, 2))(c7)
#     u8 = Concatenate()([u8, c2])
#     c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
#     c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

#     u9 = UpSampling2D((2, 2))(c8)
#     u9 = Concatenate()([u9, c1])
#     c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
#     c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

#     outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

#     model = Model(inputs=[inputs], outputs=[outputs])
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#     return model

# # Instantiate the U-Net model
# unet_model = unet_model(input_size=(256, 256, 3))
# unet_model.summary()


In [ ]:
# import os
# import tensorflow as tf

# # Define paths
# download_folder = "C:/Users/HP/Desktop/ML_HACKATHON/newimgg"  # Path to images
# batch_size = 8

# # Assuming the train_generator is already defined as per previous code
# # Train the U-Net model using generated masks directly
# unet_model.fit(
#     train_generator,
#     steps_per_epoch=len(os.listdir(download_folder)) // batch_size,  # Adjust this if images are within subdirectories
#     epochs=5,
#     callbacks=[
#         tf.keras.callbacks.ModelCheckpoint("unet_best_weights.weights.h5", save_best_only=True, save_weights_only=True),  # Updated extension to .weights.h5
#         tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
#     ]
# )
